# Assignment 4

In [1]:
import xarray as xr; from netCDF4 import Dataset, num2date
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np; import pandas as pd
import nc_time_axis
import cftime
import math
from scipy.spatial import distance

In [2]:
nodes = pd.read_csv('/projects/b1045/AoT/AoT_reduced_data_10m/nodes.csv')
sensors = pd.read_csv('/projects/b1045/AoT/AoT_reduced_data_10m/sensors.csv')
provenance = pd.read_csv('/projects/b1045/AoT/AoT_reduced_data_10m/provenance.csv')
data = pd.read_csv('/projects/b1045/AoT/AoT_reduced_data_10m/data.csv')


In [3]:
# List of node_ids
node_ids = nodes['node_id'].to_list()

In [4]:
# number of unique sensors at locations
nodes['description'].describe()

count                      91
unique                      6
top       AoT Chicago (S) [C]
freq                       58
Name: description, dtype: object

In [5]:
def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [6]:
def closest_node(point, nodes):
    closest_node = None 
    closest_node_coordinates = None
    for node in nodes: 
        distance = haversine(point, node)
        if closest_node is None or distance < closest_node :
            closest_node = distance
            closest_node_coordinates = node
    return closest_node_coordinates

In [7]:
coords = zip(nodes['lat'], nodes['lon'])
home = (42.054636, -87.6861345)

closest_node(home, coords)

(41.994597, -87.655523)

In [8]:
# Find row of closest node
nodes.loc[nodes['lat'] == 41.994597]

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp
12,001e0610b9e5,AoT_Chicago,02F,Sheridan Rd & Granville Ave Chicago IL,41.994597,-87.655523,AoT Chicago (S) [C],2017/11/28 00:00:00,NaN


In [9]:
# Time period of data availibility for node closest to me
data.loc[data['node_id'] == '001e0610b9e5']

,timestamp,node_id,subsystem,sensor,parameter,value_hrf_sum,value_hrf_count,value_hrf_average
1267577,2018/05/10 10:35:00,001e0610b9e5,lightsense,hih6130,temperature,125.010,1,125.010
1267578,2018/05/10 10:35:00,001e0610b9e5,lightsense,tmp421,temperature,-0.060,1,-0.060
1267579,2018/05/10 10:35:00,001e0610b9e5,metsense,bmp180,temperature,241.000,1,241.000
1267580,2018/05/10 10:35:00,001e0610b9e5,metsense,htu21d,temperature,128.860,1,128.860
1267581,2018/05/10 10:35:00,001e0610b9e5,metsense,pr103j2,temperature,15.550,1,15.550
1267582,2018/05/10 10:35:00,001e0610b9e5,metsense,tmp112,temperature,-254.000,1,-254.000
1267583,2018/05/10 10:35:00,001e0610b9e5,metsense,tsys01,temperature,214.750,1,214.750
6513240,2018/06/21 17:15:00,001e0610b9e5,chemsense,at0,temperature,17.090,1,17.090
6513241,2018/06/21 17:15:00,001e0610b9e5,chemsense,at1,temperature,17.310,1,17.310
6513242,2018/06/21 17:15:00,001e0610b9e5,chemsense,at2,temperature,17.700,1,17.700


In [ ]:
is_sensor = data['sensor'] == 'hih6130'
sensor_time = data[is_sensor]

plt.figure(figsize = (10,3))
plt.plot(sensor_time['timestamp'], sensor_time['value_hrf_average'])

**Questions**

1)There are 112 sensors on the AoT. There sensors include ones for CO, H2S, NO2, O3, pms at different granularity, humidity, temperature and pressure.

2)There are 91 unique node IDs in the AoT

3)Not all locations have the same set of sensors. There are 6 unique descriptions for sensor types at different locations. 

4)The node at (41.994597, -87.655523) is closest to my address at 2018 Pratt ct., Evanston 

5)The sensors at the node closest to me has a chemical sensor, 'metsense', 'audio','lightsense', 'ep', 'wagman', and 'nc'

6)The time period of data availibility from this node is 2018/05/10 10:35:00 to 2018/10/31 23:55:00

7)